<a href="https://colab.research.google.com/github/Yash0Gupta/NER-NLP/blob/main/NER_nlp_aat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit spacy transformers PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok


In [ ]:
!ngrok authtoken 2qGdjO4F02GEwfzc63jF1AUOZMk_6K36Jg92UzTnqTRwLMq1U

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
from pyngrok import ngrok

# Create the Python script file with your code
code = '''import spacy
import streamlit as st
import pandas as pd
import re
from io import StringIO
from spacy import displacy
import fitz  # PyMuPDF for PDF extraction
from transformers import pipeline

# Load SpaCy model
@st.cache_resource
def load_model():
    nlp = spacy.load("en_core_web_sm")
    return nlp

# Load summarization model
@st.cache_resource
def load_summarizer():
    summarizer = pipeline("summarization")
    return summarizer

# Initialize models
nlp = load_model()
summarizer = load_summarizer()

# Function to extract text from PDF
@st.cache_data
def extract_text_from_pdf(file):
    doc = fitz.open(stream=file.read(), filetype="pdf")
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Function to convert dataframe to CSV
def convert_df_to_csv(df):
    csv = df.to_csv(index=False)
    return StringIO(csv).getvalue()

# Streamlit app layout
def app():
    st.title("TEXTLENS")
    st.write("This app performs Named Entity Recognition (NER) and text summarization.")

    # File upload
    uploaded_file = st.file_uploader("Upload a PDF file", type=["pdf"])
    text = ""
    if uploaded_file is not None:
        text = extract_text_from_pdf(uploaded_file)
        st.write("Extracted text from the PDF:")
        st.text_area("PDF Text", text, height=300)

    # Manual text input
    else:
        text = st.text_area("Enter the text for analysis")

    # Perform operations when the Analyze button is clicked
    if st.button("Analyze"):
        if text:
            # Summarization
            with st.spinner("Generating summary..."):
                summary = summarizer(text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
            st.subheader("Text Summary")
            st.write(summary)

            # NER Analysis
            doc = nlp(text)
            entities = [(ent.text, ent.label_) for ent in doc.ents]

            # Entity Categorization
            entity_types = sorted(set([ent[1] for ent in entities]))
            selected_entity_types = st.multiselect(
                "Select entity types to extract", options=entity_types, default=entity_types
            )
            filtered_entities = [ent for ent in entities if ent[1] in selected_entity_types]

            # Display the extracted entities
            if filtered_entities:
                st.subheader("Named Entities")
                df = pd.DataFrame(filtered_entities, columns=["Entity", "Label"])
                st.dataframe(df)

                # Download button for entities
                csv = convert_df_to_csv(df)
                st.download_button("Download Entities as CSV", csv, "entities.csv", "text/csv")
            else:
                st.write("No named entities found for the selected categories.")
        else:
            st.write("Please enter some text or upload a PDF for analysis.")

# Run the Streamlit app
if __name__ == "__main__":
    app()
'''

with open("ner_app.py", "w") as f:
    f.write(code)

# Open the ngrok tunnel to expose the Streamlit app
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

# Run Streamlit in the background
os.system("streamlit run ner_app.py --server.port 8501")


Streamlit app is live at: NgrokTunnel: "https://2838-35-240-230-31.ngrok-free.app" -> "http://localhost:8501"


In [8]:
!pkill ngrok